## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
* torch.nn.AveragePool2d(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* torch.nn.AdaptiveAvgPool2d(output_size)
* torch.nn.AdaptiveMaxPool2d(output_size)

Layer de liniarizare:

* torch.nn.Flatten()



In [0]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(3,10,(3,3),(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(3,10,(13,13),(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d((3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d((5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 24, 24]
*   [1, 10, 9, 9]
*  [1, 3, 2, 2]



In [0]:
# https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d

# kerne_size = dim umbra
# stride = dim pas tranzitie ( 1 => sare cate o casuta, dar si linie)
# padding = dist intre patratelele din umbra

import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer1 = nn.Conv2d(3,10,(7,7),(4,4)) # [1, 10, 24, 24]
print("Conv1 result shape",layer1(dummy_input_tensor).shape)

layer2 = nn.Conv2d(3,10,(3,3),(11,11)) # [1, 10, 9, 9]
print("Conv1 result shape",layer2(dummy_input_tensor).shape)

layer3 = nn.AdaptiveAvgPool2d((2,2)) # [1, 3, 2, 2]
print("Global Pool result shape",layer3(dummy_input_tensor).shape)

# https://pytorch.org/docs/stable/nn.html#conv2d
# https://pytorch.org/docs/stable/nn.html#maxpool2d

Conv1 result shape torch.Size([1, 10, 24, 24])
Conv1 result shape torch.Size([1, 10, 9, 9])
Global Pool result shape torch.Size([1, 3, 2, 2])


## Instantierea seturilor de date

In [0]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

Extracting ./data/cifar-10-python.tar.gz to ./data


## Crearea Dataloader-ului

### Cerinte
 * Implementati functia de preprocesare a datelor, __preproc_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.

In [0]:
import torch
import numpy as np
import torch.utils.data as data
from torchvision.transforms.functional import to_tensor, normalize

def preproc_fn(examples):
  ### Completati codul pentru cerinta aici
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    # In linia de mai jos imaginea este normalizata astfel incat sa aiba toate valorile in 
    # [-1, 1] in loc de [0, 255]
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    vector_image = normalized_tensor_image.unsqueeze(0)
    processed_images.append(vector_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)

  return torch_images, torch_labels

train_loader = data.DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=preproc_fn)
test_loader = data.DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=preproc_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * Creati o clasa ce mosteneste din clasa nn.Module ce reprezinta o retea neurala convolutionala pentru clasificare pe 10 clase pe datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori
    * Liniarizati iesirea din cel de-al doilea strat convolutional pentru a 
    * Stratul final trebuie sa fie un strat 'fully-connected'
    * Folositi o functie de activare la alegere

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [0]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()

    self.conv_layer1 = nn.Conv2d(3,3,(6,6))
    self.conv_layer2 = nn.Conv2d(3,3,(2,2),padding=3)
    self.pool = nn.MaxPool2d((2,2))
    self.linear1 = nn.Linear(3*16*16,100)
    self.activation = nn.ReLU()
    self.linear2 = nn.Linear(100,10)

  def forward(self,x):
    x = self.conv_layer1(x)
    x = self.conv_layer2(x)
    x = self.pool(x)
    x = x.view(x.shape[0],-1)
    x = self.activation(self.linear2(self.activation(self.linear1(x))))
    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte
  * Numarul de epoci
  * Retea
  * Optimizator

In [0]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
network = Net()

# Definiti optimizatorul
optimizer = optim.Adam(network.parameters(),lr=1e-3)

# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [0]:
def train_fn(epochs: int, train_loader: data.DataLoader, test_loader: data.DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    # Iteram prin fiecare exemplu din dataset
    for images, labels in train_loader:

      # Aplicam reteaua neurala pe imaginile de intrare
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe adnotarile imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a aplica gradientii pe parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculul acuratetii
    count = len(test_loader)
    correct = 0

    for test_image, test_label in test_loader:
      out_class = torch.argmax(net(test_image))
      if out_class == test_label:
        correct += 1

    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e, (correct / count) * 100))

## Antrenarea

### Cerinte
  * Antrenati prima retea
  * Modificati celula de mai sus pentru a antrena LeNet si rulati din nou celula
  * Antrenati reteaua LeNet

In [0]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 1.9488651752471924
Acuratetea la finalul epocii 0 este 38.03%
Loss-ul la finalul epocii 1 are valoarea 1.6794464588165283
Acuratetea la finalul epocii 1 este 43.11%
Loss-ul la finalul epocii 2 are valoarea 1.5824429988861084
Acuratetea la finalul epocii 2 este 46.22%
Loss-ul la finalul epocii 3 are valoarea 1.5552605390548706
Acuratetea la finalul epocii 3 este 47.09%
Loss-ul la finalul epocii 4 are valoarea 1.4732308387756348
Acuratetea la finalul epocii 4 este 47.66%
Loss-ul la finalul epocii 5 are valoarea 1.4518003463745117
Acuratetea la finalul epocii 5 este 47.89%
Loss-ul la finalul epocii 6 are valoarea 1.4558188915252686
Acuratetea la finalul epocii 6 este 49.41%
Loss-ul la finalul epocii 7 are valoarea 1.432271957397461
Acuratetea la finalul epocii 7 este 49.67%
Loss-ul la finalul epocii 8 are valoarea 1.3424851894378662
Acuratetea la finalul epocii 8 este 49.83%
Loss-ul la finalul epocii 9 are valoarea 1.3386147022247314
Acuratetea la 

## Reteaua LeNet

### Cerinte
  * Implementati reteaua LeNet dupa figura de mai jos


![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)


In [0]:
# INPUT => CONV => RELU => POOL => CONV => RELU => POOL => FC => RELU => FC
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, (5,5))
    self.conv2 = nn.Conv2d(6,16,(5,5))
    self.linear1 = nn.Linear(16*5*5, 120)
    self.linear2 = nn.Linear(120, 84)
    self.pooling = nn.MaxPool2d((2,2))
    self.activation=nn.ReLU()

  def forward(self,x):
    x = self.conv1(x)
    x = self.activation(x)
    x = self.pooling(x)
    x = self.conv2(x)
    x = self.activation(x)
    x = self.pooling(x)

    x = x.view(x.shape[0], -1)
    x = self.linear1(x)
    x = self.activation(x)
    x = self.linear2(x)

    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [0]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 20

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=0.001)

# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
lenet_optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [0]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

Loss-ul la finalul epocii 0 are valoarea 4.4111456871032715
Acuratetea la finalul epocii 0 este 0.65%
Loss-ul la finalul epocii 1 are valoarea 4.393129348754883
Acuratetea la finalul epocii 1 este 3.96%
Loss-ul la finalul epocii 2 are valoarea 4.372461318969727
Acuratetea la finalul epocii 2 este 8.04%
Loss-ul la finalul epocii 3 are valoarea 4.346150875091553
Acuratetea la finalul epocii 3 este 8.66%
Loss-ul la finalul epocii 4 are valoarea 4.315252304077148
Acuratetea la finalul epocii 4 este 8.99%
Loss-ul la finalul epocii 5 are valoarea 4.265673637390137
Acuratetea la finalul epocii 5 este 10.10%
Loss-ul la finalul epocii 6 are valoarea 4.194132328033447
Acuratetea la finalul epocii 6 este 12.46%
Loss-ul la finalul epocii 7 are valoarea 4.024998664855957
Acuratetea la finalul epocii 7 este 11.25%
Loss-ul la finalul epocii 8 are valoarea 3.6066970825195312
Acuratetea la finalul epocii 8 este 10.02%
Loss-ul la finalul epocii 9 are valoarea 2.8639261722564697
Acuratetea la finalul epo

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* torch.nn.BatchNorm2d(num_features)
* torch.nn.InstanceNorm2d(num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de functii de activate:



* Relu
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.



  
